In [5]:
import numpy
from PIL import Image, ImageDraw
help(ImageDraw.rectangle())

AttributeError: module 'PIL.ImageDraw' has no attribute 'rectangle'

In [ ]:
help()


Welcome to Python 3.6's help utility!

If this is your first time using Python, you should definitely check out
the tutorial on the Internet at https://docs.python.org/3.6/tutorial/.

Enter the name of any module, keyword, or topic to get help on writing
Python programs and using Python modules.  To quit this help utility and
return to the interpreter, just type "quit".

To get a list of available modules, keywords, symbols, or topics, type
"modules", "keywords", "symbols", or "topics".  Each module also comes
with a one-line summary of what it does; to list the modules whose name
or summary contain a given string such as "spam", type "modules spam".

help> modules

Please wait a moment while I gather a list of all available modules...



/home/rui/v3/lib/python3.6/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated since IPython 4.0.You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)
/usr/lib/python3.6/pkgutil.py:107: VisibleDeprecationWarning: zmq.eventloop.minitornado is deprecated in pyzmq 14.0 and will be removed.
    Install tornado itself to use zmq with the tornado IOLoop.
    
  yield from walk_packages(path, info.name+'.', onerror)


IPython             backcall            jupyter_client      selectors
PIL                 base64              jupyter_console     send2trash
__future__          bdb                 jupyter_core        setuptools
_ast                binascii            keyword             shelve
_asyncio            binhex              kiwisolver          shlex
_bisect             bisect              lib2to3             shutil
_blake2             bleach              linecache           signal
_bootlocale         builtins            locale              site
_bz2                bz2                 logging             sitecustomize
_codecs             cProfile            lzma                six
_codecs_cn          calendar            macpath             smtpd
_codecs_hk          cgi                 macurl2path         smtplib
_codecs_iso2022     cgitb               mailbox             sndhdr
_codecs_jp          chunk               mailcap             socket
_codecs_kr          cmath               markupsafe

help> ImageDraw
No Python documentation found for 'ImageDraw'.
Use help() to get the interactive help utility.
Use help(str) for help on the str class.

help> PIL.ImadeDraw
No Python documentation found for 'PIL.ImadeDraw'.
Use help() to get the interactive help utility.
Use help(str) for help on the str class.

help> PIL.ImageDraw
Help on module PIL.ImageDraw in PIL:

NAME
    PIL.ImageDraw

DESCRIPTION
    # The Python Imaging Library
    # $Id$
    #
    # drawing interface operations
    #
    # History:
    # 1996-04-13 fl   Created (experimental)
    # 1996-08-07 fl   Filled polygons, ellipses.
    # 1996-08-13 fl   Added text support
    # 1998-06-28 fl   Handle I and F images
    # 1998-12-29 fl   Added arc; use arc primitive to draw ellipses
    # 1999-01-10 fl   Added shape stuff (experimental)
    # 1999-02-06 fl   Added bitmap support
    # 1999-02-11 fl   Changed all primitives to take options
    # 1999-02-20 fl   Fixed backwards compatibility
    # 2000-10-12 fl   Copy o

In [1]:
import numpy as np
from PIL import Image, ImageDraw

import torch
import torch.nn as nn
from skimage import io
import math

from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

IMG_H, IMG_W = 300, 400
# length and width of blocks (fixed for now)
block_l, block_w = 25, 25
num_images = 50

# img_list = []
true_coords = []


# Calc rectangle vertices. makeRectangle() credit Sparkler, stackoverflow, feb 17
def makeRectangle(l, w, theta, offset=(0, 0)):
    c, s = math.cos(theta), math.sin(theta)
    rectCoords = [(l/2.0, w/2.0), (l/2.0, -w/2.0), (-l/2.0, -w/2.0), (-l/2.0, w/2.0)]
    return [(c*x-s*y+offset[0], s*x+c*y+offset[1]) for (x, y) in rectCoords]


# ---- Make depth images ---
for i in range(num_images):
    orient = 0 # degrees
    img = Image.new('RGB', (IMG_H, IMG_W), 'gray')

    # block_l and _w offset so blocks don't run off edge of image
    rand_x = int(np.random.rand() * (IMG_W-block_l))
    rand_y = int(np.random.rand() * (IMG_H-block_w))

    true_coords.append(np.array((rand_x, rand_y)))

    rect_vertices = makeRectangle(block_l, block_w, orient, offset=(rand_x,
                                                                    rand_y))
    idraw = ImageDraw.Draw(img)
    idraw.polygon(rect_vertices, fill=1)

    img.save('./data/rect'+str(i)+'.png')


class RectDepthImgsDataset(Dataset):
    """Artificially generated depth images dataset"""

    def __init__(self, img_dir, coords, transform=None):
        """
        """
        self.img_dir = img_dir
        self.true_coords = coords
        self.transform = transform

    def __len__(self):
        return len(self.true_coords)

    def __getitem__(self, idx):
        # image = self.images[idx]
        image = io.imread(self.img_dir + '/rect'+str(idx)+'.png')
        image = torch.FloatTensor(image).permute(2, 0, 1)
        coords = torch.FloatTensor(true_coords[idx])

        if self.transform:
            image = self.transform(image)

        # sample = {'image': image, 'grasp': str(coords[0]) + str(coords[1])}
        sample = {'image': image, 'grasp': coords}
        sample = image, coords

        return sample


# Hyper parameters
num_epochs = 30
num_classes = 2
batch_size = 10
learning_rate = 0.001

# Dataset is depth images of rectangular blocks
train_dataset = RectDepthImgsDataset(img_dir='./data', coords=true_coords)

# Data loader
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size,
                          shuffle=True)

const = 16*97*72


class Net(nn.Module):  # CIFAR is 32x32x3, MNIST is 28x28x1)
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc1 = nn.Linear(const, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = x.view(-1, 3, 400, 300)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, const)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = Net()

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

print('Training model now...')
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i_batch, (images, labels) in enumerate(train_loader):

        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        loss.backward()
        optimizer.step()

        if (i+1) % 1 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1,
                                                                      num_epochs,
                                                                      i+1,
                                                                      total_step,
                                                                      loss.item()))


def imshow_coord(img):
    draw = ImageDraw.Draw(img)
    draw.point(xy, options)
    img.show()



Training model now...
Epoch [1/30], Step [50/5], Loss: 42008.9141
Epoch [1/30], Step [50/5], Loss: 11960.2686
Epoch [1/30], Step [50/5], Loss: 7639.7158
Epoch [1/30], Step [50/5], Loss: 12057.4707
Epoch [1/30], Step [50/5], Loss: 9673.5654
Epoch [2/30], Step [50/5], Loss: 9026.2441
Epoch [2/30], Step [50/5], Loss: 12020.9404
Epoch [2/30], Step [50/5], Loss: 16020.1377
Epoch [2/30], Step [50/5], Loss: 13845.5029
Epoch [2/30], Step [50/5], Loss: 12391.0527
Epoch [3/30], Step [50/5], Loss: 10586.3633
Epoch [3/30], Step [50/5], Loss: 12906.6182
Epoch [3/30], Step [50/5], Loss: 10045.6963
Epoch [3/30], Step [50/5], Loss: 10790.8320
Epoch [3/30], Step [50/5], Loss: 7712.9683
Epoch [4/30], Step [50/5], Loss: 6462.1162
Epoch [4/30], Step [50/5], Loss: 9409.4834
Epoch [4/30], Step [50/5], Loss: 11769.6025
Epoch [4/30], Step [50/5], Loss: 14212.6738
Epoch [4/30], Step [50/5], Loss: 8041.5259
Epoch [5/30], Step [50/5], Loss: 12505.1787
Epoch [5/30], Step [50/5], Loss: 11674.6709
Epoch [5/30], Ste

In [ ]:

# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))

# _, predicted = torch.max(outputs, 1)

# print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              # for j in range(4)))